# Classifier

In [290]:
import numpy as np
import pandas as pd
from scapy.all import *
import pyshark
import math

### Import pcap files

In [291]:
# https://scapy.readthedocs.io/en/latest/usage.html


In [292]:
# Lets first just try them all

scapy_cap = rdpcap('new_network_traces/pcap_network_traces_1_1.pcap')
for packet in scapy_cap:
    print("***** Packet")
    print("* Ether")
    # print(f"Source: {packet[Ethernet].src}")
    # print(f"Dest: {packet[IP].dst}")
    print("* IP")
    print(f"Source: {packet[IP].src}")
    print(f"Dest: {packet[IP].dst}")
    print("* TCP")
    #sport = tcp.sport
    #dport = tcp.dport
    # Sequence number, ack number
    #seq = tcp.seq
    #ack = tcp.ack
        
    # 
#     print(f"TCP Sport: {packet[TCP].sport}")
#     print(f"TCP Dport: {packet[TCP].dport}")
#     print(f"TCP Seq: {packet[TCP].seq}")
#     print(f"TCP Ack: {packet[TCP].ack}")
    
#     print(f"TIME: {packet.time}")
    
    
    # print(dir(packet))
    # print(Is(packet))
    # print(f"Summary: {packet.summary()}")
    print(f"Show: {packet.show()}")
    
    

***** Packet
* Ether
* IP
Source: 172.18.0.2
Dest: 109.105.109.162
* TCP
###[ Ethernet ]### 
  dst       = 02:42:45:87:c3:dd
  src       = 02:42:ac:12:00:02
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 576
     id        = 55816
     flags     = DF
     frag      = 0
     ttl       = 64
     proto     = tcp
     chksum    = 0xd78f
     src       = 172.18.0.2
     dst       = 109.105.109.162
     \options   \
###[ TCP ]### 
        sport     = 36784
        dport     = 60784
        seq       = 2247354939
        ack       = 79282381
        dataofs   = 5
        reserved  = 0
        flags     = PA
        window    = 65535
        chksum    = 0x8952
        urgptr    = 0
        options   = []
###[ Raw ]### 
           load      = "\x17\x03\x03\x02\x13>;\\x9a\\xcd\\xf1\\xab\\x81\x1cc랧\\xed\\xb6\\xe3\x10r\x17\\x8d\\xfe\\xfd\x1c^\\xb2\\xb7' i8w\\x84K\\x8a>m\\xfb\\xc0B\\x82V\\xf8\\xddġC\\xe9睚\\x9a+\\xa3\x0f\\xdc\\xe6\x7f\\xf

* TCP
###[ Ethernet ]### 
  dst       = 02:42:45:87:c3:dd
  src       = 02:42:ac:12:00:02
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 40
     id        = 55895
     flags     = DF
     frag      = 0
     ttl       = 64
     proto     = tcp
     chksum    = 0xd958
     src       = 172.18.0.2
     dst       = 109.105.109.162
     \options   \
###[ TCP ]### 
        sport     = 36784
        dport     = 60784
        seq       = 2247369903
        ack       = 79553177
        dataofs   = 5
        reserved  = 0
        flags     = A
        window    = 65535
        chksum    = 0x873a
        urgptr    = 0
        options   = ''

Show: None
***** Packet
* Ether
* IP
Source: 109.105.109.162
Dest: 172.18.0.2
* TCP
###[ Ethernet ]### 
  dst       = 02:42:ac:12:00:02
  src       = 02:42:45:87:c3:dd
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 15950
     id        = 

Dest: 172.18.0.2
* TCP
###[ Ethernet ]### 
  dst       = 02:42:ac:12:00:02
  src       = 02:42:45:87:c3:dd
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 12730
     id        = 27589
     flags     = 
     frag      = 0
     ttl       = 63
     proto     = tcp
     chksum    = 0x5759
     src       = 109.105.109.162
     dst       = 172.18.0.2
     \options   \
###[ TCP ]### 
        sport     = 60784
        dport     = 36784
        seq       = 79703071
        ack       = 2247377943
        dataofs   = 5
        reserved  = 0
        flags     = PA
        window    = 65535
        chksum    = 0xb8cc
        urgptr    = 0
        options   = []
###[ Raw ]### 
           load      = ' SI\x1d4\\xd4hN\\x8f\\xec\\x89*\x04\\xa3C2\\xe0\\x8c\\xa4?\x1f\\xd3 \\xa4W\\xb7\\x8eI`\\xfdj/\\x89{\\xa32\\xb1R\\xbd\\xb4\\xe3\\xb4K\\xceZ.\\xb4!ΌI{ּ\\x8a\\xe4T\\xa5O\\xbc\\xd6\\xcaaa\\xbfd\\xf1\x03V\\xc8\\xd7\\xd6\\xf0\x1c\\x9d\\xd2-\x02n\\xb

In [293]:
packets_ip = []
packets_ip_sender = []
packets_ip_receiver = []
packets_time = []
packets_length = []
# NOTE: This is just one packet
scapy_cap = rdpcap('new_network_traces/pcap_network_traces_1_1.pcap')
for packet in scapy_cap:
    src = packet[IP].src
    dst = packet[IP].dst
    
    # print(f"Src: {src}")
    # print(f"Dst: {dst}")
    
    packets_ip.append(src)
    packets_ip.append(dst)
    
    packets_ip_sender.append(src)
    packets_ip_receiver.append(dst)
    
    packets_time.append(packet.time)
    packets_length.append(packet[IP].len)

src = packets_ip[0]
dst = packets_ip[1]

### Extract Features

In [294]:
# 1. number of packets statistics
num_packets = len(packets_ip)

# number of incoming and outgoing packets
# 2. incoming and outgoing packets as a fraction of total packets
out_fraction = packets_ip.count(src)/num_packets
in_fraction = packets_ip.count(dst)/num_packets

In [295]:
# 3. Packet ordering statistics 
def get_ordering_statistics(packets_ip, ip):
    '''
    Packet ordering statistics
    For each successive incoming and outgoing packet, 
    the total number of packets seen before it in the sequence
    
    Returns mean, standard deviation
    '''
    order_counts = []
    for i in range(len(packets_ip)):
        i_ip = packets_ip[i]

        # print(f"I: {i}")
        # print(f"I IP: {i_ip}")
        if (i_ip == ip):
            count = 0
            for j in range(i+1, len(packets_ip)-i-1):
                j_ip = packets_ip[j]
                # print(f"J: {j}")
                # print(f"J IP: {j_ip}")
                if (i_ip == j_ip):
                    count+=1
                else:
                    break

            # print(f"Count: {count}")
            if (count > 0):
                order_counts.append(count)

    order_counts = np.array(order_counts)

    # mean, standard deviation
    order_counts_mean = order_counts.mean()
    order_counts_std = order_counts.std()
    
    return order_counts_mean, order_counts_std

# Get mean, standard deviation for packet ordering statistics
src_order_mean, src_order_std = get_ordering_statistics(packets_ip, src)
dst_order_mean, dst_order_std = get_ordering_statistics(packets_ip, dst)

In [296]:
# 4. Concentration of outgoing packets
# Packet sequence split into non-overlapping chunks of 20 packets
# Count the number of outgoing packets in each of the chunks
# Along with entire chunk sequence, extract the mean, median, max, and standard deviation
def conc_outgoing_packets(packets_ip, src):
    outgoing_conc = []
    start = 0
    while (start < len(packets_ip) - 1):
        end = start + 20
        if (end < len(packets_ip) - 1):
            this_chunk = packets_ip[start:end]
            this_count = this_chunk.count(src)
            outgoing_conc.append(this_count)

        start += 20

    outgoing_conc = np.array(outgoing_conc)

    # Total mean, max, standard deviation, third quartile
    outgoing_conc_mean = outgoing_conc.mean()
    outgoing_conc_median = np.percentile(outgoing_conc, [50])[0]
    outgoing_conc_max = outgoing_conc.max()
    outgoing_conc_std = outgoing_conc.std()
    
    return outgoing_conc_mean, outgoing_conc_median, outgoing_conc_max, outgoing_conc_std

# Obtain concentration statistics of outgoing packets
outgoing_conc_mean, outgoing_conc_median, outgoing_conc_max, outgoing_conc_std = conc_outgoing_packets(packets_ip, src)


In [297]:
# 5. Concentration of incoming and outgoing packets in first and last 30 packets
def conc_30(packets_ip, ip):
    first_30 = packets_ip[:30]
    last_30 = packets_ip[-30:]

    first_fraction = first_30.count(ip)/30
    last_fraction = last_30.count(ip)/30
    
    return first_fraction, last_fraction

# Obtain first 30/last 30 fraction for src and dst
first_fraction_src, last_fraction_src = conc_30(packets_ip, src)
first_fraction_dst, last_fraction_dst = conc_30(packets_ip, dst)

In [298]:
# 6. Number of packets per second
def packets_per_second(packets_ip, packets_time):
    total_time = packets_time[-1] - packets_time[0]
    packets_per_second = num_packets / total_time
    
    return packets_per_second

# Obtain packets per second
packets_ps = packets_per_second(packets_ip, packets_time)

In [299]:
# 7. Alternative concentration features

In [300]:
# 8. Packet inter-arrival time statistics
# for the total, incoming packet streams, outgoing packet streams
# extract the lists of inter-arrival times between packets
# For each list, extract the max, mean, st_dev, third quartile

def total_interarrival_times(packets_time):
    # Determine total interarrival times
    total_interarrival_times = []
    for i in range(len(packets_time) - 2):
        total_interarrival_times.append(packets_time[i+1] - packets_time[i])

    # Convert to numpy array
    total_interarrival_times = np.array(total_interarrival_times)
    
    # Total mean, max, standard deviation, third quartile
    ia_mean = total_interarrival_times.mean()
    ia_max = total_interarrival_times.max()
    ia_std = total_interarrival_times.std()
    ia_tq = np.percentile(total_interarrival_times, [75])[0]
    
    return ia_mean, ia_max, ia_std, ia_tq

total_mean, total_max, total_std, total_tq = total_interarrival_times(packets_time)
    
def ip_interarrival_times(packets_ip, packets_time, ip):
    
    # Determine ip times
    times = []
    for i in range(len(packets_time)):
        if (packets_ip[i*2] == ip):
            times.append(packets_time[i])
    
    # Determine interarrival_times
    ia_times = []
    for i in range(len(times) - 2):
        ia_times.append(times[i+1] - times[i])
    
    # Convert to numpy array
    ia_times = np.array(ia_times)
    
    # mean, max, standard deviation, third quartile
    ia_mean = ia_times.mean()
    ia_max = ia_times.max()
    ia_std = ia_times.std()
    ia_tq = np.percentile(ia_times, [75])[0]
    
    return ia_mean, ia_max, ia_std, ia_tq
    
src_mean, src_max, src_std, src_tq = ip_interarrival_times(packets_ip, packets_time, src)
dst_mean, dst_max, dst_std, dst_tq = ip_interarrival_times(packets_ip, packets_time, dst)

In [301]:
# Get length features
def total_length(packets_length):
    packets_length = np.array(packets_length)
    
    t_mean = packets_length.mean()
    t_median = np.percentile(packets_length, [50])[0]
    t_min = packets_length.min()
    t_max = packets_length.max()
    t_std = packets_length.std()

    return t_mean, t_median, t_min, t_max, t_std

total_len_mean, total_len_median, total_len_min, total_len_max, total_len_std = total_length(packets_length)

In [302]:
def ip_length(packets_ip, packets_ip_sender, packets_length, ip):
    # Determine ip lengths
    lengths = []
    for i in range(len(packets_length)):
        if (packets_ip_sender[i] == ip):
            lengths.append(packets_length[i])
    
    packets_length = np.array(packets_length)
    
    l_mean = packets_length.mean()
    l_median = np.percentile(packets_length, [50])[0]
    l_min = packets_length.min()
    l_max = packets_length.max()
    l_std = packets_length.std()

    return l_mean, l_median, l_min, l_max, l_std

src_len_mean, src_len_median, src_len_min, src_len_max, src_len_std = ip_length(packets_ip, packets_ip_sender, packets_length, src)
dst_len_mean, dst_len_median, dst_len_min, dst_len_max, dst_len_std = ip_length(packets_ip, packets_ip_sender, packets_length, dst)

In [303]:
# scapy_cap = rdpcap('network_traces/pcap_network_traces_1.pcap')
def extract_packet_features(grid, pcap_file):
    '''
    Extracts all features for a pcap file
    '''
    packets_ip = []
    packets_ip_sender = []
    packets_ip_receiver = []
    packets_time = []
    packets_length = []

    scapy_cap = rdpcap(pcap_file)
    for packet in scapy_cap:
        src = packet[IP].src
        dst = packet[IP].dst

        # print(f"Src: {src}")
        # print(f"Dst: {dst}")

        packets_ip.append(src)
        packets_ip.append(dst)

        packets_ip_sender.append(src)
        packets_ip_receiver.append(dst)

        packets_time.append(packet.time)
        packets_length.append(packet[IP].len)
        
    if (len(packets_ip) == 0):
        return None

    src = packets_ip[0]
    dst = packets_ip[1]

    # 1. number of packets statistics
    num_packets = len(packets_ip)

    # number of incoming and outgoing packets
    # 2. incoming and outgoing packets as a fraction of total packets
    out_fraction = packets_ip.count(src)/num_packets
    in_fraction = packets_ip.count(dst)/num_packets

    # 3. packet ordering statistics
    src_order_mean, src_order_std = get_ordering_statistics(packets_ip, src)
    dst_order_mean, dst_order_std = get_ordering_statistics(packets_ip, dst)

    # 4. concentration statistics of outgoing packets
    outgoing_conc_mean, outgoing_conc_median, outgoing_conc_max, outgoing_conc_std = conc_outgoing_packets(packets_ip, src)

    # 5. concentration of first 30/last 30 fraction for src and dst
    first_fraction_src, last_fraction_src = conc_30(packets_ip, src)
    first_fraction_dst, last_fraction_dst = conc_30(packets_ip, dst)

    # 6. packets per second
    packets_ps = packets_per_second(packets_ip, packets_time)

    # 7. packet interarrival times
    total_mean, total_max, total_std, total_tq = total_interarrival_times(packets_time)
    src_mean, src_max, src_std, src_tq = ip_interarrival_times(packets_ip, packets_time, src)
    dst_mean, dst_max, dst_std, dst_tq = ip_interarrival_times(packets_ip, packets_time, dst)
    
    # 8. packet length features
    total_len_mean, total_len_median, total_len_min, total_len_max, total_len_std = total_length(packets_length)
    src_len_mean, src_len_median, src_len_min, src_len_max, src_len_std = ip_length(packets_ip, packets_ip_sender, packets_length, src)
    dst_len_mean, dst_len_median, dst_len_min, dst_len_max, dst_len_std = ip_length(packets_ip, packets_ip_sender, packets_length, dst)
    
    # Features array
    features = [
        grid,
        num_packets, 
        out_fraction, in_fraction, 
        src_order_mean, src_order_std, dst_order_mean, dst_order_std,
        outgoing_conc_mean, outgoing_conc_median, outgoing_conc_max, outgoing_conc_std,
        first_fraction_src, last_fraction_src, first_fraction_dst, last_fraction_dst,
        packets_ps,
        total_mean, total_max, total_std, total_tq,
        src_mean, src_max, src_std, src_tq,
        dst_mean, dst_max, dst_std, dst_tq,
        total_len_mean, total_len_median, total_len_min, total_len_max, total_len_std,
        src_len_mean, src_len_median, src_len_min, src_len_max, src_len_std,
        dst_len_mean, dst_len_median, dst_len_min, dst_len_max, dst_len_std
               ]
    
    return features
    

### Extract features for all pcap files

In [304]:
column_names = [
    'grid',
    '1_num_packets', 
    '2_out_fraction', '2_in_fraction', 
    '3_src_order_mean', '3_src_order_std', '3_dst_order_mean', '3_dst_order_std',
    '4_outgoing_conc_mean', '4_outgoing_conc_median', '4_outgoing_conc_max', '4_outgoing_conc_std',
    '5_first_fraction_src', '5_last_fraction_src', '5_first_fraction_dst', '5_last_fraction_dst',
    '6_packets_ps',
    '7_ia_total_mean', '7_ia_total_max', '7_ia_total_std', '7_ia_total_tq',
    '7_ia_src_mean', '7_ia_src_max', '7_ia_src_std', '7_ia_src_tq',
    '7_ia_dst_mean', '7_ia_dst_max', '7_ia_dst_std', '7_ia_dst_tq',
    '8_total_len_mean', '8_total_len_median', '8_total_len_min', '8_total_len_max', '8_total_len_std',
    '8_src_len_mean', '8_src_len_median', '8_src_len_min', '8_src_len_max', '8_src_len_std',
    '8_dst_len_mean', '8_dst_len_median', '8_dst_len_min', '8_dst_len_max', '8_dst_len_std'
]

features_df = pd.DataFrame(columns=column_names)
features_df

,grid,1_num_packets,2_out_fraction,2_in_fraction,3_src_order_mean,3_src_order_std,3_dst_order_mean,3_dst_order_std,4_outgoing_conc_mean,4_outgoing_conc_median,...,8_src_len_mean,8_src_len_median,8_src_len_min,8_src_len_max,8_src_len_std,8_dst_len_mean,8_dst_len_median,8_dst_len_min,8_dst_len_max,8_dst_len_std


In [305]:
# OLD FILES
def extract_all_packets(num_packets, features_df):
    '''
    Extracts all pcap files
    '''
    for i in range(1, num_packets + 1):
        # Obtain file
        # print(f"I: {i}")
        this_pcap_file = f'network_traces_old/pcap_network_traces_{i}.pcap'
        # print(f"This File: {this_pcap_file}")

        # Extract features
        grid = math.floor((i-1)/10) + 1
        this_features = extract_packet_features(grid, this_pcap_file)
        this_features_df = pd.Series(this_features, index=features_df.columns)

        # Add to features_d
        features_df = features_df.append(this_features_df, ignore_index=True)
        
    return features_df

In [310]:
# NEW FILES
def extract_all_packets(num_grids, num_packets, features_df):
    '''
    Extracts all pcap files
    '''
    for i in range(1, num_grids + 1):
        print(f"I: {i}")
        for j in range(1, num_packets + 1):
            # Obtain file
            # print(f"I: {i}")
            this_pcap_file = f'new_network_traces/pcap_network_traces_{i}_{j}.pcap'
            # print(f"This File: {this_pcap_file}")

            # Extract features
            grid = math.floor((i-1)/10) + 1
            
            try:
                this_features = extract_packet_features(grid, this_pcap_file)
            except:
                print(f"Exception: {i}, {j}")
            else:
                if (this_features != None):
                    this_features_df = pd.Series(this_features, index=features_df.columns)
                    # Add to features_d
                    features_df = features_df.append(this_features_df, ignore_index=True)
        
    return features_df

In [311]:
all_features = extract_all_packets(100, 100, features_df)

I: 1
I: 2
I: 3
I: 4
I: 5
I: 6
I: 7
I: 8
I: 9
I: 10
I: 11
I: 12
I: 13
I: 14
I: 15
I: 16
I: 17
I: 18
I: 19


/Users/michaelgysel/.pyenv/versions/3.7.2/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: Mean of empty slice.
/Users/michaelgysel/.local/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Exception: 19, 30
I: 20
I: 21
I: 22
I: 23
I: 24
I: 25
I: 26
I: 27
I: 28
I: 29
I: 30
I: 31
I: 32
I: 33
I: 34
I: 35
I: 36
I: 37
I: 38
I: 39
I: 40
I: 41
I: 42
I: 43
I: 44
I: 45
I: 46
I: 47
I: 48
I: 49
I: 50
I: 51
I: 52
I: 53
I: 54
I: 55
I: 56
I: 57
I: 58
I: 59
I: 60
I: 61
Exception: 61, 1
Exception: 61, 2
Exception: 61, 3
Exception: 61, 5
Exception: 61, 6
Exception: 61, 7
Exception: 61, 8
Exception: 61, 9
Exception: 61, 10
Exception: 61, 11
Exception: 61, 12


/Users/michaelgysel/.pyenv/versions/3.7.2/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: Mean of empty slice.
/Users/michaelgysel/.local/lib/python3.7/site-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/Users/michaelgysel/.local/lib/python3.7/site-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/Users/michaelgysel/.local/lib/python3.7/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Exception: 61, 13
Exception: 61, 14
Exception: 61, 15
Exception: 61, 16
Exception: 61, 17
Exception: 61, 18
Exception: 61, 20
Exception: 61, 21
I: 62
I: 63
I: 64
I: 65
I: 66
I: 67
I: 68
I: 69
I: 70
I: 71
I: 72
I: 73
I: 74
I: 75
I: 76
I: 77
I: 78
I: 79
I: 80
Exception: 80, 99
I: 81
I: 82
I: 83
I: 84
I: 85
I: 86
Exception: 86, 49
I: 87
I: 88
I: 89
I: 90
I: 91
I: 92
Exception: 92, 98
I: 93
I: 94
I: 95
I: 96
I: 97
I: 98
I: 99
I: 100


In [312]:
all_features

,grid,1_num_packets,2_out_fraction,2_in_fraction,3_src_order_mean,3_src_order_std,3_dst_order_mean,3_dst_order_std,4_outgoing_conc_mean,4_outgoing_conc_median,...,8_src_len_mean,8_src_len_median,8_src_len_min,8_src_len_max,8_src_len_std,8_dst_len_mean,8_dst_len_median,8_dst_len_min,8_dst_len_max,8_dst_len_std
0,1,578,0.5,0.5,1.0,0.0,1.0,0.0,10.0,10.0,...,1706.705882,576.0,40,16960,3020.695687,1706.705882,576.0,40,16960,3020.695687
1,1,486,0.5,0.5,1.0,0.0,1.0,0.0,10.0,10.0,...,1787.769547,576.0,40,18580,3266.634701,1787.769547,576.0,40,18580,3266.634701
2,1,502,0.5,0.5,1.0,0.0,1.0,0.0,10.0,10.0,...,1692.501992,576.0,40,22280,3154.195843,1692.501992,576.0,40,22280,3154.195843
3,1,534,0.5,0.5,1.0,0.0,1.0,0.0,10.0,10.0,...,1599.498127,576.0,40,18328,2978.708762,1599.498127,576.0,40,18328,2978.708762
4,1,544,0.5,0.5,1.0,0.0,1.0,0.0,10.0,10.0,...,1621.757353,576.0,40,18336,2926.996681,1621.757353,576.0,40,18336,2926.996681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9956,10,1108,0.5,0.5,1.0,0.0,1.0,0.0,10.0,10.0,...,1076.498195,576.0,40,13652,1687.135880,1076.498195,576.0,40,13652,1687.135880
9957,10,1066,0.5,0.5,1.0,0.0,1.0,0.0,10.0,10.0,...,1087.797373,480.0,40,11320,1750.556351,1087.797373,480.0,40,11320,1750.556351
9958,10,970,0.5,0.5,1.0,0.0,1.0,0.0,10.0,10.0,...,1189.513402,576.0,40,9910,1707.339695,1189.513402,576.0,40,9910,1707.339695
9959,10,1002,0.5,0.5,1.0,0.0,1.0,0.0,10.0,10.0,...,1153.960080,576.0,40,11320,1796.490408,1153.960080,576.0,40,11320,1796.490408


In [313]:
all_features.to_csv('features.csv')